In [1]:
import sys,os
import torch 
import torch.nn as nn
from torchvision.models import resnet50
from torch.optim import Adam
from torch.utils.data import DataLoader
sys.path.append("../src")

In [2]:
import utils.config as cfg
from datasets.landmark_dataset import LandmarkDataset
from utils.models_utils import TrainTransforms,ModelUtils 

Mean and std values are specified for ImageNet deviation. *temp solution*, needs to be determined for DeepFashion dataset

In [3]:
#MEAN & STD declaration
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

In [4]:
INITIAL_LR = 1e-4
NUM_EPOCHS = 20
BATCH_SIZE = 16

Memory allocation

In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
PIN_MEMORY = True if DEVICE == "cuda" else False # to speed up loading data on CPU to training it on GPU
# see https://discuss.pytorch.org/t/when-to-set-pin-memory-to-true/19723

Loading dataset

In [6]:

tf_list = TrainTransforms(mean=MEAN,std=STD)

Compose(
    RandomHorizontalFlip(p=0.5)
    RandomVerticalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [7]:
dataset = LandmarkDataset(root="../"+cfg.LANDMARK_DATASET_PATH, runing_mode="train",transforms_list=tf_list )
train_dataloader = DataLoader(dataset=dataset,batch_size=BATCH_SIZE,num_workers=os.cpu_count())

In [8]:
# resnet = resnet50()
resnet_utils = ModelUtils()
model =resnet_utils.build_model(model = resnet50,pretrained=True,gradation=False)

[INFO]loading model with pretrained weigts.
[INFO] Freezing all hiden layers.


/home/filippatyk/anaconda3/envs/clothes/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/filippatyk/anaconda3/envs/clothes/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
